# 01 PIV Snapshot to HDF - Advanced topic

In [the first PIV Snapshot](01_snapshot_pivview.ipynb) a simple conversion is shown. However, this is only the basic conversion. Oftentimes, a user may want to adjust some details every time the conversion is performed. For this, "user defined operations" (UDO) can be injected to the conversion process.

In this example, we want to inject an operation, which assigns [standard_names](https://matthiasprobst.github.io/ssno/) to some of the variables:

In [ ]:
from piv2hdf import PIVSnapshot

from piv2hdf import tutorial
from piv2hdf.pivview import PIVViewNcFile
from piv2hdf import UserDefinedHDF5Operation

from datetime import datetime

## Define the UDP:

A UDO is a class defined by `UserDefinedHDF5Operation`, which enforces the implementation of `__call__(self, h5:h5tbx.File)`, where `h5` is an opened HDF5 file.

In [ ]:
from typing import Dict
import h5rdmtoolbox as h5tbx

class MyUDO(UserDefinedHDF5Operation):

    def __init__(self, sn_translation: Dict):
        self.sn_translation = sn_translation

    def __call__(self, h5:h5tbx.File):
        for k, v in h5.items():
            sn = self.sn_translation.get(k)
            if sn:
                v.attrs["standard_name"] = sn

In [ ]:
pivview_nc_file = tutorial.PIVview.get_snapshot_nc_files()[0]

Instantiate and **inject the UDO** during the instantiation of your `PIVFile`:

In [ ]:
my_translations = {
    "x": "x_coordinate",
    "y": "y_coordinate",
    "u": "x_velocity",
    "v": "y_velocity"
}

In [ ]:
my_udo = MyUDO(my_translations)

In [ ]:
pivfile = PIVViewNcFile(
    filename=pivview_nc_file,
    parameter_filename=None,
    user_defined_hdf5_operations=my_udo
)

Convert to HDF5 as in the previous example:

In [ ]:
recording_dtime = datetime(2023, 1, 15, 13, 42, 2, 3)
snapshot = PIVSnapshot(piv_file=pivfile, recording_dtime=recording_dtime)
hdf_filename = snapshot.to_hdf()

Let's check if it worked:

In [ ]:
with h5tbx.File(hdf_filename) as h5:
    h5["x"].dump()